# Dinamično programiranje

Dinamično programiranje je algoritmičen pristop, ki je podoben pristopu deli in vladaj. Tudi pri uporabi dinamičnega programiranja bomo razbili problem na manjše podprobleme, poiskali optimalne rešitve podproblemov in si z njimi pomagali pri rešitvi začetnega problema. Pomembne lastnosti problema, pri katerem si lahko pomagamo z dinamičnim programiranjem so:
- neodvisnost podproblemov: Posamezen podproblem lahko rešujemo neodvisno od drugih podproblemov.
- optimalna podstruktura: Optimalna rešitev problema vsebuje optimalne rešitve podproblemov.
- **prekrivanje/ponavljanje podproblemov**: To je glavna lastnost, ki jo bomo izkoristili za izboljšave in v čemer se pristop razlikuje od tehnike deli in vladaj.

Tehniko lahko enostavno povzamemo z nasvetom "ne računaj enakih stvari večkrat", v praksi pa je kljub temu nekoliko bolj zapleteno - kako to doseči, katere stvari sploh so enake, ...

Pristop nima nobene veze z dinamično alokacijo pomnilnika. Poimenoval ga je njen avtor Richard Bellman. "Programiranje" se nanaša na reševanje optimizacijskega problema, poodobno kot matematično programiranje/optimizacija. Pridevnik "dinamično" pa se nanaša na različne podprobleme.

## Fibonaccijevo zaporedje

Osnovno idejo dinamičnega programiranja si oglejmo na trivialnem primeru Fibonacijevega zaporedja, ki je definirano rekurzivno kot: $F_0 = 0,\quad F_1 = 1,\quad F_n = F_{n-1}+F_{n-2}$. Zanima nas $n$-to število v zaporedju. Pri večjih $n$-jih bodo vrednosti zaporedja precej velike, vendar se s tem ne bomo ukvarjali in bomo zadovoljni z rezultatom, ki je posledica preliva (*overflow*).

In [13]:
int f(int n) {
    if (n<=0) return n;
    return f(n-1)+f(n-2);
}

SyntaxError: invalid syntax. Perhaps you forgot a comma? (699695227.py, line 1)

In [14]:
for (int n=0;n<10;n++) {
    cout << n << ": " << f(n) << endl;
}

SyntaxError: invalid syntax. Perhaps you forgot a comma? (382516576.py, line 1)

Vrednosti izgledajo pravilne. Hitro pa ugotovimo, da na ta način ne bomo mogli računati vrednosti že za malo večje $n$-je. Težava je v eksponentni velikosti drevesa rekurzivnih klicev. Listov tega drevesa, kjer je rezultat funkcije 1, je natanko $F_n$. Poleg tega pa imamo še liste z vrednostjo 0 in vsa notranja vozlišča. Skratka, ogromno število vozlišč oz. klicev funkcije.

In [15]:
cout << f(100) << endl;  // prepocasi

SyntaxError: invalid syntax (2096509138.py, line 1)

Opazimo lahko, da se bo funkcija izvedla večkrat z istim argumentom $n$. Če se nismo kje zmotili, bi moral imeti vsak tak klic funkcije tudi enak rezultat. Rezultat si lahko ob prvem klicu funkcije shranimo, v kasnejših klicih pa ga samo vrnemo. To je pristop **od zgoraj navzdol** (*top-down*), ki je znan tudi pod imenom **memoizacija** (*memoization*, brez "r"). Funkcija se bo torej za vsak možen argument izvedla natanko enkrat, ob ostalih klicih pa bo takoj vrnila vrednost, česar niti ne bomo šteli kot klic funkcije. Število klicev funkcije bo torej $O(n)$, čas izvedbe posameznega klica funkcije pa $O(1)$. Rešitev ima časovno in prostorsko zahtevnost $O(n)$.

Za ugotavljanje, ali je bil nek podproblem že rešen ali ne, lahko v tem primeru izkoristimo kar vrednost 0, saj bomo kot izračunane rezultate vpisovali samo večja števila. V splošnem pa bi lahko imeli eno tabelo, ki bi nam povedala, ali je bil nek podproblem že rešen, ter drugo tabelo, ki bi hranila dejanske rezultate. Zaradi enostavnosti bomo uporabili dovolj veliko fiksno tabelo dovolj. Namesto tega bi lahko uporabili katerokoli implementacijo slovarja, ki bi imel kot ključ argumente, ki predstavljajo opis podproblema, za pripadajočo vrednost pa njegovo rešitev.

In [16]:
const int N=10000;
int memo[N+1];  // memoizacijska tabela

int f(int n) {
    if (n<=0) return n;
    if (memo[n]!=0) return memo[n];
    memo[n]=f(n-1)+f(n-2)
    return memo[n];
}

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3478077152.py, line 1)

Če smo malo bolj sistematični, lahko rešujemo podprobleme v takem vrstnem redu, da imamo rešitve manjših podproblemov vedno že rešene, ko jih potrebujemo. Podprobleme bomo torej reševali od manjših proti večjim, kar v tem primeru pomeni od manjših proti večjim $n$-jem. Takemu reševanju rečemo **od spodaj navzgor** (*bottom-up*). Časovna in prostorska zahtevnost sta enaki kot v prejšnjem primeru, le da sta še bolj očitni.

In [17]:
int fib[N+1];

fib[0]=0;
fib[1]=1;
for (int n=2;n<=N;n++) fib[n]=fib[n-1]+fib[n-2];
cout << fib(100) << endl;

SyntaxError: invalid syntax. Perhaps you forgot a comma? (705820079.py, line 1)

Zaradi sistematičnosti pa smo lahko malo bolj prostorsko učinkoviti. Vedno namreč potrebujemo rezultate samo zadnjih dveh izračunanih problemov. Tako lahko prostorsko zahtevnost zmanjšamo na $O(1)$.

In [18]:
int f(int n) {
    int f1=0, f2=1;
    for (int i=2;i<=n;i++) {
        int fi=f1+f2;
        f2=f1;
        f1=fi;
    }
}

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2796297425.py, line 1)

In [19]:
cout << f(100) << endl;

NameError: name 'cout' is not defined

## Žabji skoki

Vzdolž potoka gleda iz vode $n$ skal na koordinatah $x_1 < x_2 < ... < x_n$. Žabec sedi na prvi skali in bi rad z zaporedjem skokov po skalah prispel do zadnje skale. V enem skoku lahko skoči najmanj $a$ in največ $b$ enot daleč v smeri proti cilju. Kakšno je najmanjše število skokov, ki jih potrebuje za to?

Če je $a=0$, smo že v poglavju o požrešnih algoritmih na podobnem problemu ugotovili, da lahko z vsakim skokom skoči do najbolj oddaljene skale, ki jo še doseže, in bo s tem minimiziral število svojih skokov. Vpeljava spodnje meje dolžine skoka pa problem zakomplicira.

Če razmišljamo rekurzivno, se bo žabec v prvem skoku premaknil na neko skalo $x_i$, ki je oddaljena med $a$ in $b$ od skale $x_1$. Če take skale sploh ni, pot do cilja ne obstaja. Za to je porabil en skok, nato pa se mora v čim manjšem številu skokov premakniti s skale $x_i$ do cilja. Definirajmo podproblem $f(i)$ kot najmanjše število skokov, ki ga žabec potrebuje, da pride na cilj z $i$-te skale:
- $f(n) = 0$
- $f(i) = \min_{j>i:\;a \leq x_j-x_i \leq b\;} (1 + f(j))$

Očitno bo prišlo do ponavljanja podproblemov. Do neke skale lahko žabec pride na več načinov, ampak za optimalno pot od tam do cilja je povsem nepomembno, kako je do tja prišel. Pomembno je samo, na kateri skali se nahaja. Zato si lahko rešitev shranimo in jo kasneje po potrebi uporabimo, ne da bi jo računali ponovno. Lahko pa bi probleme reševali tudi sistematično po principu od spodaj navzgor, kar v tem primeru pomeni od skal bližje cilju proti tistim bližje začetku.

Rešiti moramo $O(n)$ podproblemov, za rešitev vsakega od njih pa moramo preveriti $O(n)$ možnosti za naslednji skok. Časovna zahtevnost je $O(n^2)$, prostorska pa $O(n)$.

In [29]:
const int inf=1e9;

int f(int i, vector<int> &x) {
    if (i==n) return 0;
    int best=inf;
    for (int j=i+1;j<=n;j++) {
        int d=x[j]-x[i];
        if (a<=d && d<=b) best=min(best, 1+f(j,x));
    }
    return best;
}

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3020718178.py, line 1)

In [30]:
vector<int> x = {0,1,3,4,6,10}  // 0 na prvem mestu je namenjena zgolj poravnavi indeksov
cout << f(1,x) << endl;

SyntaxError: cannot assign to comparison (3018260832.py, line 1)

## Rezanje palice

Pri problemu rezanja palice (*rod cutting*) imamo podano palico dolžine $n$, ki jo želimo razrezati na manjše kose in te kose prodati posamično za čim večjo skupno ceno. Dolžina palice in dolžine kosov morajo biti celoštevilske. Podano imamo tabelo cen $c$, v kateri nam $i$-to število $c_i$ pove, za kakšno ceno bomo lahko prodali palico dolžine $i$. Daljši kot je kos, za večjo ceno ga bomo lahko prodali: veljalo bo $c_i \leq c_{i+1}$. Kakšen je največji možen izkupiček od prodaje razrezane palice?

Oglejmo si primer s spodnjo tabelo cen:
<table>
<tr style="border: 1px solid"><td>$i$</td><td>1</td><td>2</td><td>3</td><td>4</td><td>5</td><td>6</td><td>7</td><td>8</td></tr>
<tr style="border: 1px solid"><td>$c_i$</td><td>2</td><td>5</td><td>7</td><td>9</td><td>10</td><td>17</td><td>17</td><td>20</td></tr>
</table>

Naj bo dolžina palice $n=8$:
- Če jo razrežemo na dva kosa dolžin 3 in 5, bomo dobili samo $c_3 + c_5 = 18$.
- Če razrežemo palico na kose dolžine 1, bomo zanjo dobili $n \cdot c_1 = 16$.

Če bi bila dolžina palice $n=4$, pa je optimalna rešitev razrez na polovico $c_2 + c_2 = 10$.

Rekurzivni razmislek o zaslužku $f(n)$ pri optimalnem rezanju palice dolžine $n$ nam pove, da bomo morali izbrati dolžino prvega reza. Če je palica dolžine $n$, si moramo izbrati enega od rezov dolžine $x \leq n$ (s čimer zaslužimo $c_x$) ter optimalno zrezati preostanek palice dolžine $n-x$. Ker ne vemo, katera dolžina reza bo najboljša, rekurzivno preverimo vse. Uporabimo tokrat pristop od spodaj navzgor in izračunajmo zaslužke za vedno daljše palice: $f(n) = \max_{x \leq n} f(n-x)+c_x$.

In [1]:
vector<int> c = {0,2,5,7,9,10,17,17,20};
int N=8;
int f[N+1];
f[0]=0;
for (int n=1;n<=N;n++) {
    f[n]=0;
    for (int x=1;x<=n;x++) {
        f[n]=max(f[n], f[n-x]+c[x]);
    }
}
cout << f[8] << endl;
cout << f[4] << endl;

SyntaxError: cannot assign to comparison (3454708363.py, line 1)

Časovna zahtevnost algoritma je $O(n^2)$, prostorska pa $O(n)$.

Razmislimo še o **rekonstrukciji** rešitve. Katere reze je treba narediti, da dosežemo optimalno ceno? Za vsak podproblem poiščemo potezo, ki je vodila do optimalnega rezultata. Druga možnost pa je, da si že ob reševanju podproblema shranimo optimalno potezo: npr. v dodatni tabeli $g(n)$ bi lahko hranili $x$, pri katerem funkcija $f(n)$ doseže svoj maksimum.

In [2]:
int n=8;
while (n>0) {
    for (int x=1;x<=n;x++) {
        if (f[n]==f[n-x]+c[x]) {
            cout << x << endl;
            n-=x;
            break;
        }
    }
}

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1313276284.py, line 1)

## Pot v mreži

V labirintu višine $h$ in širine $w$ oz. tabeli znakov '.', ki predstavljajo prosto polje in '#', ki predstavlajo blokirano polje, nas zanima, na koliko načinov lahko pridemo iz levega-zgornjega kota v desni-spodnji kot, pri čemer se lahko premikamo samo desno in navzdol. V spodnjem primeru obstajajo tri take poti.

```text
.#....
....#.
.#..#.
......
```

Rekurzivno bi problem reševali tako, da bi se s trenutne celice poskusili premakniti desno in navzdol (če sta oba premika sploh možna) in sešteli možne poti do cilja iz nove lokacije (sosednje celice). Dosedanji problemi so imel eno-dimenzionalen opis podproblema, kjer smo podproblem opisali z eno spremenljivko. Tokrat pa podproblem opišemo z dvema dimenzijama - vrstico in stolpcem celice. Če je polje zasedeno ali se nahaja izven mreže, je število poti do cilja enako $0$, sicer pa velja $f(i,j) = f(i+1,j) + f(i,j+1)$. Robni pogoj v desnem-spodnjem kotu je $f(h-1, w-1)=0$.

Podprobleme lahko rešujemo sistematično po vrsticah od spodaj navzgor in znotraj vrstice od desne proti levi. Tako imamo potrebne rešitve podproblemov vsakič že na voljo. Časovna in prostorska zahtevnost sta $O(hw)$.

In [4]:
vector<string> lab = {".#....",
                      "....#.",
                      ".#..#.",
                      "......"};
int h=lab.size(), w=lab[0].size();
int f[h+1][w+1]={};  // inicializacija na privzeto vrednost 0
for (int i=h-1;i>=0;i--) {
    for (int j=w-1;j>=0;j--) {
        if (i==h-1 && j==w-1) f[i][j]=1;
        else if (f[i][j]=='#') f[i][j]=0;
        else f[i][j]=f[i+1][j]+f[i][j+1];
    }
}
cout << f[0][0] << endl;

SyntaxError: cannot assign to comparison (460281349.py, line 1)

Prostorsko zahtevnost bi lahko izboljšali na $O(w)$, ker pri računanju vrednosti $f(i,*)$ potrebujemo samo že izračunane rezultate desno v isti vrstici $f(i,*)$ in eno vrstico nižje $f(i+1,*)$.

## Najdaljše skupno podzaporedje

Pri problemu najdaljšega skupnega podzaporedja (*longest common subsequence, LCS*) nizov $S$ in $T$ (dolžine $n$ in $m$), iščemo najdaljši niz $\text{LCS}(S,T)$, ki se pojavi kot podzaporedje (ne nujno podniz) v $S$ in v $T$. Oglejmo si primer $S=A\bar{B}\bar{C}\bar{B}D\bar{A}B$ in $T=\bar{B}D\bar{C}A\bar{B}\bar{A}$, kjer je eno izmed najdaljših skupnih podzaporedij $\text{LCS}(S,T)=BCBA$ dolžine 4.

Drugačen pogled na isti problem je poravnava obeh nizov, da se pri tem čim več znakov ujema.
```
AB C BDAB
 BDCAB A
```
Rekurzivni razmislek je sledeč:
- Če se oba niza začneta z enakim znakom, je ta znak lahko začetek LCS-ja. Preostanek pa je LCS za en znak krajših nizov.
- Če se niza razlikujeta v prvem znaku, potem vsaj en od teh dveh znakov ne bo del LCS-ja. Preizkusimo obe možnosti in rešimo problem z nizoma, kjer je en malo krajši.

Naj bo $\text{LCS}(i,j)$ najdaljši skupni podniz nizov $S_i S_{i+1} \ldots S_{n-1}$ in $T_j T_{j+1} \ldots T_{m-1}$:
$$\text{LCS}(i,j)=\max \begin{cases}
1+\text{LCS}(i+1,j+1) & \text{če } S_i = T_j \\
\text{LCS}(i+1,j) \\
\text{LCS}(i,j+1) \\
\end{cases}\\
$$
Robni primeri pa so $\text{LCS}(n,*) = 0$ in $\text{LCS}(*,m) = 0$.


Problem lahko rešujemo sistematično od večjih proti manjšim $i$-jem in enako za $j$. Rešujemo torej probleme z vedno daljšimi priponami nizov $S$ in $T$. S tem pravzaprav izpolnjujemo 2D tabelo od desnega spodnjega kota proti levemu zgornjemu, tako da izberemo večjo od spodnje in desne celice. Če sta začetna znaka enaka, pa upoštevamo še diagonalen rezultat povečan za 1. Dokažemo lahko tudi, da bo ta diagonalna poteza vedno optimalna, če je na voljo.

In [ ]:
int LCS(string s, string t) {
    int n=s.size(), m=t.size();
    int lcs[n][m]={};  // inicializacija na 0
    for (int i=n-1;i>=0;i--) {
        for (int j=m-1;j>=0;j--) {
            lcs[i][j]=max(lcs[i+1][j], lcs[i][j+1]);
            if (s[i]==t[j]) lcs[i][j]=max(lcs[i][j], 1+lcs[i+1][j+1]);
        }
    }
    for (int i=0;i<n;i++) {
        for (int j=0;j<m;j++) {
            cout << lcs[i][j] << '\t';
        }
        cout << endl;
    }
    return lcs[0][0];
}

Časovna in prostorska zahtevnost sta $O(nm)$. Problem lahko rešujemo tudi v obratni smeri od konca proti začetkom nizov, kjer se vprašamo, kaj se bo zgodilo z zadnjima znakoma obeh nizov (namesto prvima), kar boste pogosto videli v drugih virih.

Kako pa bi problem rešili za tri nize? $\text{LCS}(S,T,U)$ namreč ni enak $\text{LCS}(\text{LCS}(S,T),U)$! Stanje bi opisali s trojico indeksov $\text{LCS}(i,j,k)$ in obravnavali primere podobno kot za dva niza. Če velja $S_i = T_j = U_k$, je ta znak lahko del LCS-ja, sicer pa vsaj en izmed njih ne bo in lahko enega od nizov skrajšamo.

Soroden problem je iskanje najdaljšega skupnega podniza (ne podzaporedja; *longest common substring*), kjer mora biti pojavitev podniza strnjena v obeh nizih. Ta problem ima drugačne in bolj učinkovite rešitve.

## Nahrbtnik

Podoben problem je coin change

## Najdaljše naraščajoče podzaporedje

(*longest increasing subsequence*)